# Анализ сервиса для чтения книг по подписке
Мы являемся крупной компанией, которая на фоне новых тендеций возникших в мире на волне коронавируса, решила приобрести крупный сервис для чтения книг по подписке. Нам предоставлена база данных, которая содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг

##### Цель исследования
Целью исследования является анализ данных, который поможет сформулировать ценностное предложение для нового продукта

##### Ход исследования
1. Подключение к базе данных и изучение информации, которая в ней содержится
2. Анализ данных
    - Посчитайте, сколько книг вышло после 1 января 2000 года;
    - Для каждой книги посчитайте количество обзоров и среднюю оценку;
    - Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
    - Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
    - Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
3. Написание общего вывода

##  1. Подключение к базе данных и изучение информации, которая в ней содержится

Для начала подключимся к базе данных:

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

def get_sql_query_data(query):
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    return pd.io.sql.read_sql(query, con = engine)

Просмотрим что содержится в таблице book. Выведем первые 5 строк

In [3]:
query_book = '''
select *
from books
limit 5
'''

get_sql_query_data(query_book)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query_dates = '''
select 
    min(publication_date) as min_date_publicated
    ,max(publication_date) as max_date_publicated
from books
'''

get_sql_query_data(query_dates)

,min_date_publicated,max_date_publicated
0,1952-12-01,2020-03-31


Как можно заметить, подключение к базе данных выше сработало успешно

Итак, можно заметить, что таблица book содержит:
1. book_id - ID книги
2. author_id - ID автора
3. title - наименование книги
4. num_pages - количество страниц в книге
5. publication_date - дата когда книга была опубликована
6. publisher_id - ID издателя

В базе хранятся информация о книгах выпущенных с 1 декабря 1952 года по 31 марта 2020 года

Просмотрим что содержится в таблице authors. Также выведем первые 5 строк:

In [5]:
query_authors = '''
select *
from authors
limit 5
'''
get_sql_query_data(query_authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Как можно заметить, таблица содержит информацию об авторе, а именно: 
1. author_id - ID автора
2. author - Имя автора/авторов

Изучим таблицу publishers:

In [6]:
query_publishers = '''
select *
from publishers
limit 5
'''

get_sql_query_data(query_publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Как можно заметить, таблица содержит информацию об издательствах, а именно:
1. publisher_id - ID издетельства
2. publisher - Наименование издательства

Изучим таблицу ratings:

In [7]:
query_ratings = '''
select *
from ratings
limit 5
'''

get_sql_query_data(query_ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query_rat = '''
select 
    distinct(rating) as points
from ratings
order by points
'''

get_sql_query_data(query_rat)

,points
0,1
1,2
2,3
3,4
4,5


Как можно увидеть таблица содерижит информацию как пользователь оценил ту или иную книгу. Можно увидеть что таблица содержит колонки:
1. rating_id - ID рейтинга
2. book_id - ID книги
3. username - Имя пользователя
4. rating - оценка, которую поставил пользователь (стоит отметить, что пользовательская оценка может варьировать от 1 до 5 баллов, судя по информации, выведенной выше)

Изучим последнюю таблицу reviews:

In [9]:
query_reviews = '''
select *
from reviews
limit 5
'''

get_sql_query_data(query_reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Судя по всему, данная таблица содержит информацию о пользовательских обзорах на книгу. А именно, в таблице содержаться такие колонки как:
1. review_id - ID обзора
2. book_id - ID книги
3. username - Имя пользователя
4. text - Текст обзора

### Промежуточный вывод по шагу 1:
В результате проделанной работы на шаге 1, удалось подключиться к предоставляемой базе и выявить, что:
1. База данных, хранит в себе 5 таблиц: books - информация о книге, authors - содержит информацию об авторах, publishers - содержит информацию об издательствах, ratings - содержит информацию об оценке, которую выставил пользователь книге, reviews - содержит информацию о пользовательских обзорах на книги.
2. Пользователь может проставить оценку только по 5-бальной системе баллов
3. В базе хранится информация по книгам, выпущенным с 1 декабря 1952 года по 31 марта 2020 года

## 2. Анализ данных

Посчитайте, сколько книг вышло после 1 января 2000 года;

In [10]:
query_task_1 = '''
select 
    count(distinct(book_id)) as count_books
from books
where publication_date > '2000-01-01'
'''

get_sql_query_data(query_task_1)

,count_books
0,819


С 1 января 2000 года по 31 марта 2020 года (так как выше было выявлено что это максимальная дата публикации книги) было выпущено порядка 819 книг. Это достаточно малое количество выпущенных книг за 20 лет (можно считать что выпускалось порядка 40 книг в год). Считаю, что нужно увеличить данных показатель для того чтобы привлечь клиентов от конкурентов, которые выпускают современные бестселлеры

Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [11]:
query_task2 = '''
select 
        bk.title
        ,brew.count_otz
        ,br.avg_raitings
from books as bk
left join (select 
                    book_id
                    ,round(avg(rating), 3) as avg_raitings
            from ratings
            group by book_id
            ) as br on br.book_id = bk.book_id
left join (select 
                    book_id
                    ,count(text) as count_otz
            from reviews
            group by book_id
            ) brew on brew.book_id = bk.book_id
order by brew.count_otz desc
'''

get_sql_query_data(query_task2)

,title,count_otz,avg_raitings
0,The Natural Way to Draw,NaN,3.000
1,Disney's Beauty and the Beast (A Little Golden...,NaN,4.000
2,Essential Tales and Poems,NaN,4.000
3,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.667
4,Leonardo's Notebooks,NaN,4.000
...,...,...,...
995,Stone of Farewell (Memory Sorrow and Thorn #2),1.0,5.000
996,Debt of Honor (Jack Ryan #7),1.0,3.000
997,Color: A Natural History of the Palette,1.0,4.333
998,Winter Prey (Lucas Davenport #5),1.0,4.500


Судя по полученным данным, не все пользователи сопровождают оценку книги отзывов. Однако если рассматривать книги, у которых есть и отзывы и оценки, лидируют по количеству отзывов книга сумерки. Однако большое количество отзывов не гарантируют высокий показатель средней пользовательской оценки. 

Если судить по жанру, то можно сказать что в топ-20 преобладает фентэзи/фантастика. Судя по всему карантин сказался на предпочтениях пользователей, которые в такой сложный период хотели бы сбежать в новый необычный мир. Считаю, что следует выпускать больше книг таких жанров и впредь. Также могу порекомендовать добавить книги научного жанра, а также книги по саморазвитию (т.к. сейчас период карантина, люди могут захотеть начать саморазвиваться)

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [12]:
query_task3 = '''
select 
        p.publisher
        ,cb.count_books
from publishers as p
inner join (select 
                    publisher_id
                    ,count(distinct(book_id)) as count_books
            from books
            where num_pages >50
            group by publisher_id
        ) as cb on cb.publisher_id = p.publisher_id
order by cb.count_books desc
limit 1
'''

get_sql_query_data(query_task3)

,publisher,count_books
0,Penguin Books,42


Издательство, которое выпустило самое наибольшее количество книг (у которых страниц было больше 50) стало Penguin Books, выпустившее 42 книги

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
query_task4 = '''
select 
        au.author
        ,round(avg(br.avg_rating), 2) as rating
from books as bk
inner join (select 
                    book_id
                    ,avg(rating) as avg_rating
            from ratings
            group by book_id
            having count(rating)>=50
            ) as br on br.book_id = bk.book_id
inner join authors as au on au.author_id = bk.author_id
group by au.author
order by rating desc
limit 1
'''

get_sql_query_data(query_task4)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.28


Самый популярный и обсуждаемый автор - Дж.Роулинг, которая написала серию книг о Гарри Поттере. У нее самая высокая средняя оценка равная 4.28. Считаю, что нужно и дальше добавлять книги данного автора, поскольку ее книги также и входят в топ-20 самых оцениваемых книг

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [14]:
query_task5 = '''
with users AS (
    select 
        username
        ,count(text) AS count_rev
    from 
        reviews
    where 
        username in (
                    select
                        username
                    from ratings
                    group by username
                    having count(rating) > 48
        )
    group by username
)

select
    round(avg(count_rev), 2) AS avg_rev
from users;
'''

get_sql_query_data(query_task5)

,avg_rev
0,24.0


Среди пользователей, которые поставили более 48 оценок, среднее количество написанных обзоров составляет 24, что является достаточно малым показателем. Считаю, что внедрение системы поощрения за обзоры может стимулировать пользователей к активному обмену рецензиями, что в свою очередь обогатит опыт чтения и поможет другим пользователям выбирать более интересные книги

### Промежуточные выводы по шагу 2
1. С 1 января 2000 года по 31 марта 2020 года (так как выше было выявлено что это максимальная дата публикации книги) было выпущено порядка 819 книг. Это достаточно малое количество выпущенных книг за 20 лет (можно считать что выпускалось порядка 40 книг в год). Считаю, что нужно увеличить данных показатель для того чтобы привлечь клиентов от конкурентов, которые выпускают современные бестселлеры
2. Судя по полученным данным, не все пользователи сопровождают оценку книги отзывов. Однако если рассматривать книги, у которых есть и отзывы и оценки, лидируют по количеству отзывов книга сумерки. Однако большое количество отзывов не гарантируют высокий показатель средней пользовательской оценки.  
3. Если судить по жанру, то можно сказать что в топ-20 преобладает фентэзи/фантастика. Судя по всему карантин сказался на предпочтениях пользователей, которые в такой сложный период хотели бы сбежать в новый необычный мир. Считаю, что следует выпускать больше книг таких жанров и впредь. Также могу порекомендовать добавить книги научного жанра, а также книги по саморазвитию (т.к. сейчас период карантина, люди могут захотеть начать саморазвиваться)
4. Издательство, которое выпустило самое наибольшее количество книг (у которых страниц было больше 50) стало Penguin Books, выпустившее 42 книги
5. Самый популярный и обсуждаемый автор - Дж.Роулинг, которая написала серию книг о Гарри Поттере. У нее самая высокая средняя оценка равная 4.28. Считаю, что нужно и дальше добавлять книги данного автора, поскольку ее книги также и входят в топ-20 самых оцениваемых книг
6. Среди пользователей, которые поставили более 48 оценок, среднее количество написанных обзоров составляет 24, что является достаточно малым показателем. Считаю, что внедрение системы поощрения за обзоры может стимулировать пользователей к активному обмену рецензиями, что в свою очередь обогатит опыт чтения и поможет другим пользователям выбирать более интересные книги

## Общий вывод:
Мы являемся крупной компанией, которая на фоне новых тендеций возникших в мире на волне коронавируса, решила приобрести крупный сервис для чтения книг по подписке. Нам предоставлена база данных, которая содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Целью исследования является анализ данных, который поможет сформулировать ценностное предложение для нового продукта

В результате проделанной работы на шаге 1, удалось подключиться к предоставляемой базе и выявить, что:
1. База данных, хранит в себе 5 таблиц: books - информация о книге, authors - содержит информацию об авторах, publishers - содержит информацию об издательствах, ratings - содержит информацию об оценке, которую выставил пользователь книге, reviews - содержит информацию о пользовательских обзорах на книги.
2. Пользователь может проставить оценку только по 5-бальной системе баллов
3. В базе хранится информация по книгам, выпущенным с 1 декабря 1952 года по 31 марта 2020 года

В ходе анализа данных было выявлено, что:
1. С 1 января 2000 года по 31 марта 2020 года (так как выше было выявлено что это максимальная дата публикации книги) было выпущено порядка 819 книг. Это достаточно малое количество выпущенных книг за 20 лет (можно считать что выпускалось порядка 40 книг в год). Считаю, что нужно увеличить данных показатель для того чтобы привлечь клиентов от конкурентов, которые выпускают современные бестселлеры
2. Судя по полученным данным, не все пользователи сопровождают оценку книги отзывов. Однако если рассматривать книги, у которых есть и отзывы и оценки, лидируют по количеству отзывов книга сумерки. Однако большое количество отзывов не гарантируют высокий показатель средней пользовательской оценки. 
3. Если судить по жанру, то можно сказать что в топ-20 преобладает фентэзи/фантастика. Судя по всему карантин сказался на предпочтениях пользователей, которые в такой сложный период хотели бы сбежать в новый необычный мир. Считаю, что следует выпускать больше книг таких жанров и впредь. Также могу порекомендовать добавить книги научного жанра, а также книги по саморазвитию (т.к. сейчас период карантина, люди могут захотеть начать саморазвиваться)
4. Издательство, которое выпустило самое наибольшее количество книг (у которых страниц было больше 50) стало Penguin Books, выпустившее 42 книги
5. Самый популярный и обсуждаемый автор - Дж.Роулинг, которая написала серию книг о Гарри Поттере. У нее самая высокая средняя оценка равная 4.28. Считаю, что нужно и дальше добавлять книги данного автора, поскольку ее книги также и входят в топ-20 самых оцениваемых книг
6. Среди пользователей, которые поставили более 48 оценок, среднее количество написанных обзоров составляет 24, что является достаточно малым показателем. Считаю, что внедрение системы поощрения за обзоры может стимулировать пользователей к активному обмену рецензиями, что в свою очередь обогатит опыт чтения и поможет другим пользователям выбирать более интересные книги